In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
from spacy.lang.en.stop_words import STOP_WORDS

In [3]:
import spacy
import re
import string

In [ ]:
stopwords = nltk.download('stopwords')
stopwords

In [4]:
import spacy
import re
import string
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import en_core_web_sm

In [5]:
import en_core_web_lg

In [83]:
news_df = pd.read_csv('final_data.csv')

In [84]:
news_df.columns

Index(['Unnamed: 0', 'hrefs', 'headline', 'briefing', 'body', 'category'], dtype='object')

In [85]:
news_df.drop('Unnamed: 0',axis=1,inplace=True)

In [86]:
news_df.tail(10)

,hrefs,headline,briefing,body,category
20994,https://gate.ahram.org.eg/News/4122898.aspx,رئيس البورصة: متوسطات العائد على الاستثمار في ...,قال رامي الدكاني رئيس البورصة المصرية، إن سوق ...,"Rami El Dokani, Chairman of the Egyptian Stock...",اقتصاد
20995,https://gate.ahram.org.eg/News/4122861.aspx,اهتمام خليجي كبير بالشركات المصرية المطروحة.. ...,عقد الوفد المصري الذي يضم القيادات التنفيذية ل...,"The Egyptian delegation, which includes the ex...",اقتصاد
20996,https://gate.ahram.org.eg/News/4122808.aspx,وزيرة التعاون الدولي تستقبل نائب رئيس الوزراء ...,استقبلت الدكتورة رانيا المشاط، وزيرة التعاون ا...,"Dr. Rania Al-Mashat, Minister of International...",اقتصاد
20997,https://gate.ahram.org.eg/News/4122801.aspx,الفيومي: الرقابة الإلكترونية على السلع تسهم في...,أشاد النائب الدكتور محمد عطية الفيومي، رئيس غر...,"Representative Dr. Mohamed Attia Al-Fayoumi, P...",اقتصاد
20998,https://gate.ahram.org.eg/News/4122730.aspx,وزيرة التخطيط: معظم مشروعات الدولة تستهدف تنمي...,شاركت الدكتورة هالة السعيد، وزيرة التخطيط والت...,شاركت الدكتورة هالة السعيد، وزيرة التخطيط والت...,اقتصاد
20999,https://gate.ahram.org.eg/News/4122711.aspx,سعر الذهب اليوم الإثنين في مصر.. عيار 21 يسجل ...,سجل سعر الذهب اليوم الإثنين في مصر استقرارا في...,"The price of gold recorded today, Monday, in E...",اقتصاد
21000,https://gate.ahram.org.eg/News/4122703.aspx,سعر اليورو اليوم الإثنين 20 فبراير 2023 في الس...,شهد سعر اليورو اليوم الإثنين، استقرارًا ملحوظً...,"Today, Monday, the price of the euro witnessed...",اقتصاد
21001,https://gate.ahram.org.eg/News/4122674.aspx,وفد من الغرف التجارية يبحث سبل توفير عمالة مصر...,قال وزير القوى العاملة حسن شحاتة، إنه بصدد توق...,"Minister of Manpower, Hassan Shehata, said tha...",اقتصاد
21002,https://gate.ahram.org.eg/News/4122669.aspx,​الضرائب: ورش العمل لتقديم الدعم الفني للممولي...,أعلن مختار توفيق، رئيس مصلحة الضرائب المصرية، ...,"Mokhtar Tawfiq, Head of the Egyptian Tax Autho...",اقتصاد
21003,https://gate.ahram.org.eg/News/4122662.aspx,وزارة التموين: ارتفاع معدل توريد القصب إلى 3 م...,أكدت شركة السكر والصناعات التكاملية التابعة لل...,The Sugar and Integrated Industries Company of...,اقتصاد


In [ ]:
news_df.category.value_counts()

In [96]:

import arabic_reshaper
from bidi.algorithm import get_display

def drop_arabic_entries(df):

    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        # Convert the text to Unicode and reshape it using arabic_reshaper
        text = row['body']
        reshaped_text = arabic_reshaper.reshape(text)

        # Check if the reshaped text contains any Arabic characters
        if any('\u0600' <= char <= '\u06FF' for char in reshaped_text):
            # If the text contains Arabic characters, drop the row from the dataframe
            df.drop(index, inplace=True)

    # Reset the index of the dataframe
    df.reset_index(drop=True, inplace=True)

    return df

In [92]:
arabic_pattern = re.compile('[\u0600-\u06FF]+')

# Define a function to check if a string contains Arabic text
def contains_arabic(text):
    return bool(arabic_pattern.search(text))

In [93]:
news_df['text'] = news_df['body'].apply(contains_arabic)

In [99]:
news_df['text'].sum()

0

In [98]:
news_df.shape

(20230, 6)

In [103]:
news_df.drop('category',axis=1,inplace=True)

In [106]:
news_df.drop('text',axis=1,inplace=True)

In [107]:
news_df.columns

Index(['hrefs', 'headline', 'briefing', 'body'], dtype='object')

In [125]:
max(news_df.new_body.tolist(), key=len)

"Engineer Tarek ElMolla Minister Petroleum Mineral Resources stress importance petrochemical industry project ministry ' strategy give project support maximize product revenue capability light distinguished product offer meet need local market support development effort Egypt he also thank company ' management employee the outstanding performance achieve company year this come attendance meeting General Assembly Egyptian Company Production ethylene derivative ETHYDCO via video conference approve result company business year for part Eng Mohamed Abdel Moneim Saleh President ETHIDCO review important business result for year explaining provide outstanding performance standard whether term production sale despite condition global economy impose RussianUkrainian crisis consequent decrease demand polyethylene product global price fierce competition companys product face international product Engineer Mohamed Abdel Moneim Saleh thank Eng Tarek El Molla Minister Petroleum Mineral Resources ful

In [7]:
nlp = spacy.load('en_core_web_lg')

nlp = en_core_web_lg.load()


In [ ]:
nltk.download('punkt')

In [113]:
def remove_punc(news_text):
    result = re.sub(r'[0-9]+', '', news_text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub('', result)
    return text

In [14]:

# Load the stop words
stop_words = set(stopwords.words('english'))


In [114]:
def remove_stop(the_text):
  # Tokenize the text into words
  words = word_tokenize(the_text)

  # Filter out the stop words from the text
  filtered_words = [word for word in words if not word in stop_words]

  # Join the filtered words into a string
  filtered_text = ' '.join(filtered_words)

  # return the filtered text
  return(filtered_text)

In [115]:
def lemm(text):
    lemme=[]
    for token in nlp(text):
        lemme.append(token.lemma_)

    return " ".join(lemme)

In [116]:
def preprocessing_text(text):
    text = remove_punc(text)
    text = remove_stop(text)
    text = lemm(text)
    return text

In [122]:
news_df['new_body'] = news_df['body'].apply(preprocessing_text)

In [123]:
news_df.head(5)

,hrefs,headline,briefing,body,new_body
0,https://gate.ahram.org.eg/News/4272955.aspx,محافظ الغربية يتابع الاستعدادات النهائية لامتح...,التقى الدكتور طارق رحمي محافظ الغربية، بالمهند...,"Dr. Tariq Rahmi, Governor of Gharbia, met with...",Dr Tariq Rahmi Governor Gharbia meet Eng Nasse...
1,https://gate.ahram.org.eg/News/4272956.aspx,"""الشباب والرياضة"" بالأقصر تنفذ ندوات عن التوعي...",نفذت إدارة تنمية الشباب بمديرية الشباب والرياض...,The Department of Youth Development at the Dir...,the Department Youth Development Directorate Y...
2,https://gate.ahram.org.eg/News/4272936.aspx,متابعة عمل الوحدات الصحية والمراكز الطبية في 7...,قامت إدارة المتابعة بمديرية الشئون الصحية بمحا...,"Today, Friday, the Follow-up Department of the...",today Friday Followup Department Directorate H...
3,https://gate.ahram.org.eg/News/4272934.aspx,اعتماد ملعب مركز شباب الصلعا كأول ملعب لـ«المي...,قام أحمد سمير، رئيس الاتحاد المصرى للميني فوتب...,"Ahmed Samir, President of the Egyptian Federat...",Ahmed Samir President Egyptian Federation Mini...
4,https://gate.ahram.org.eg/News/4272941.aspx,إخماد حريق فى أرض مزروعة بالقمح في مركز الحامو...,تمكنت قوات الحماية المدنية بمركز الحامول فى كف...,The Civil Protection Forces at Al-Hamoul Cente...,the Civil Protection Forces AlHamoul Center Ka...


In [126]:
news_df.drop('body',axis=1,inplace=True)

In [127]:
news_df.columns

Index(['hrefs', 'headline', 'briefing', 'new_body'], dtype='object')

In [147]:
news_df.to_csv('app_data.csv')

In [128]:
# Initialize a TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000 , ngram_range=(1, 3) )

In [130]:
tfidf_vectorizer.fit(news_df.new_body)

TfidfVectorizer(max_features=5000, ngram_range=(1, 3))

In [214]:
from joblib import dump, load

In [211]:
with open('lda.pkl', 'rb') as f:
    lda = pickle.load(f)

In [237]:
news_df.new_body[2]

'today Friday Followup Department Directorate Health Affairs Kafr ElSheikh Governorate head Muhammad Ramadan Jaadr Director Department visit health unit affiliate administration Fowah Riyadh AlHamoul Bella Sidi Salem Desouq Qulin affiliate medical center ensure regular work in unit presence doctor worker availability medicine necessary treat emergency case part continuous effort follow medical service provide health sector Kafr ElSheikh for seven center Health Department Nursing School Riyadh Salqa Unit Salamiyah Unit affiliate Fowah Center Medical Center Saffron Unit Health Administration Hamoul Bahnasi Units AlMasry Infantry Unit AlNouri Unit Health Administration Sidi Salem Center Monitoring progress work Hazek unit file department health department Pella medical center joint laboratory Desouk health department Shabrato unit Sarwa unit file department health department Qalin center inspect general situation unit term infrastructure equipment availability medicine supply presence man

In [271]:
rania = tfidf_vectorizer2.transform([news_df.new_body[19997]])

In [272]:
ml.predict(rania)

array([2], dtype=int64)

In [131]:
# Fit and transform the news article texts
tfidf_matrix = tfidf_vectorizer.transform(news_df.new_body)

In [138]:
input_article = news_df.new_body[22]

In [132]:
news_df.new_body[22]

'Edfu Marsa Alam road witness car rollover accident border Aswan governorate today Friday result injury people people immediately ambulance dispatch transfer injure Edfu General Hospital necessary medical examination conduct Citizens the necessary report issue incident Public Prosecution notify begin investigation order reach reason behind'

In [134]:
# Define a function to recommend similar articles
def recommend_similar_articles(article_text, tfidf_matrix=tfidf_matrix, news_df=news):
    
    # Transform the input article text into a TF-IDF vector
    article_tfidf = tfidf_vectorizer.transform([article_text])

    # Compute the cosine similarity between the input article and all other articles
    cosine_similarities = cosine_similarity(article_tfidf, tfidf_matrix)

    # Get the index of the most similar article
    similar_article_index = cosine_similarities.argmax()
  
    recommended_article_indices = cosine_similarity(tfidf_matrix[similar_article_index], tfidf_matrix).argsort()[0][-6:-1][::-1]

    return recommended_article_indices

In [139]:
# Test the function by recommending similar articles for a sample article

recommended_articles = recommend_similar_articles(input_article)
print(recommended_articles)

[16337 15036   319   151 14628]


In [142]:
news_df.new_body[319]

'today Suez Road witness microbus accident result injury people separate injury injure transfer emergency department Suez General Hospital receive necessary treatment in Suez microbus overturn injury immediately director ambulance facility direct ambulance accident site provide necessary first aid transfer injure hospital treatment it find microbus involve overturn accident accident result injury a force direct from secure road inspect accident find microbus car carry people head Cairo Suez overturn accident result injured people transfer emergency department Suez General Hospital necessary examination rumor conduct determine health status injure person a report accident competent authority begin investigate circumstance cause accident trace accident remove road'

In [ ]:
news.headline[recommended_articles[0]]

In [45]:
import googletrans

In [39]:
from googletrans import Translator

In [36]:
# Set up the translator
translator = Translator()

In [40]:
translation = translator.translate(cell, src='ar', dest='en')
print(translation.text)

The boy was playing ball in the field with his friends until midnight, then his mother called him and told him to come home.


In [ ]:

# Define the two texts to compare
text1 = news.body[22]
text2 = news.iloc[369]['body']
# Initialize a TfidfVectorizer
tfidf_vectorizer5 = TfidfVectorizer()

# Fit and transform the texts
tfidf_matrix5 = tfidf_vectorizer5.fit_transform([text1, text2])

# Compute the cosine similarity between the two texts
cosine_similarities5 = cosine_similarity(tfidf_matrix5[0], tfidf_matrix5[1])

# Print the cosine similarity score
print("Cosine similarity score:", cosine_similarities5[0][0])

In [ ]:
%%writefile re.py

import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import en_core_web_lg
import spacy
import re
import string
import nltk
import pkg_resources
pkg_resources.require("googletrans==3.1.0a0")
import googletrans
from googletrans import Translator
import pickle

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# ------------------------------------------------------------------------------------------------------

# Set the page configuration
st.set_page_config(page_title="News Recommender App", page_icon=":guardsman:", 
                   layout="wide", initial_sidebar_state="expanded")

# Define the theme settings
st.markdown("""
<style>
body {
    font-family: 'Segoe UI', sans-serif;
    color: #333333;
    background-color: #483D8B;
}
h1 {
    font-size: 48px;
    color: #0072b2;
}
h2 {
    font-size: 36px;
    color: #009e73;
}
</style>
""", unsafe_allow_html=True)

st.sidebar.success("Select a tool above.")

# ----------------------------------------------------------------------------------------------------------- #
# defining a fuction to remove punctuations
def remove_punc(news_text):
    result = re.sub(r'[0-9]+', '', news_text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub('', result)
    return text

# defining a function to remove stop words
def remove_stop(the_text):
  # Tokenize the text into words
  words = word_tokenize(the_text)

  # Filter out the stop words from the text
  filtered_words = [word for word in words if not word in stop_words]

  # Join the filtered words into a string
  filtered_text = ' '.join(filtered_words)

  # return the filtered text
  return(filtered_text)

# prepare spacy model for lemmatization
nlp = spacy.load('en_core_web_lg')
nlp = en_core_web_lg.load()


# defining lemmatization function
def lemm(text):
    lemme=[]
    for token in nlp(text):
        lemme.append(token.lemma_)

    return " ".join(lemme)


# defining a fuction that combines all preprocessing steps needed
def preprocessing_text(text):
    text = remove_punc(text)
    text = remove_stop(text)
    text = lemm(text)
    return text

# ------------------------------------------------------------------------------------------------------ #

def main():
    
    # Adding the caching decorator
    @st.cache_data  
    def load_data():
        df = pd.read_csv('app_data.csv')
        return df


    # Load news articles data from CSV file
    news_df = load_data()


    # Define the title and text of the web app
    st.title("News Recommender App")
    st.subheader("Enter an article to find similar news articles")

    # Set up the translator
    translator = Translator()

    # Define the input field for the article text
    input_text = st.text_input("Article text")

    # Translate the text
    process = preprocessing_text(input_text)

    # translated text preprocessing
    try:
        translation = translator.translate(process , src='ar', dest='en')
        article_text = translation.text
    except:
        pass
# ________________________________________________________________________________________________________________ #
    # Define a button to recommend similar articles
    try:
        if st.button("Recommend"):
            # Initialize a TfidfVectorizer
            tfidf_vectorizer = TfidfVectorizer( max_features=5000, ngram_range=(1, 3) )

            # Fit and transform the news article texts
            tfidf_matrix = tfidf_vectorizer.fit_transform(news_df['new_body'])
            
            # Transform the input article text into a TF-IDF vector
            article_tfidf = tfidf_vectorizer.transform([article_text])

            # Compute the cosine similarity between the input article and all other articles
            cosine_similarities = cosine_similarity(article_tfidf, tfidf_matrix)

            # Get the index of the most similar article
            similar_article_index = cosine_similarities.argmax()

            # Get the titles of the top 5 recommended articles

            recommended_article_indices = cosine_similarity(tfidf_matrix[similar_article_index], tfidf_matrix).argsort()[0][-6:-1][::-1]
            # XGB classifier
            with open('trained_model.pkl', 'rb') as f:
                ml = pickle.load(f)
            
            
            categ = ml.predict( article_tfidf )[0]
            catgs = {"0":"محافظات" ,
                     "1" : "رياضة",
                     "2" : "عرب و عالم",
                     "3": "أخبار" ,
                     "4" : "ثقافة وفنون",
                     "5" : "حوادث",
                     "6" : "إقتصاد"}
            
            
            # Print the recommended articles
            st.header("Recommended articles:")
            
            
            for order,indx in list(enumerate(recommended_article_indices)):
                st.write(order+1 , "- " + news_df.iloc[indx]['headline'])
                st.write("Brief:     ", news_df.iloc[indx]['briefing'])
                st.write("Go to: ", news_df.iloc[indx]['hrefs'])
                st.write(" -----/----/-----/-----/-----/-----/------/-----/-----/-----/------ ")

            st.write("Classification of News: ", catgs[str(categ)])
    except:
        st.subheader("Please enter the news text")
    
    
if __name__ == '__main__':
    main()

In [158]:
news_df.columns

Index(['hrefs', 'headline', 'briefing', 'new_body'], dtype='object')